# ImmoEliza Data Analysis

## Data Cleaning

### Import Necessary Librairies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.options.mode.chained_assignment = None

### Setting the dataset in a variable called "data"

In [ ]:
data = pd.read_json("final_dataset.json")
pd.options.display.max_columns = 35             #just extending the number  of columns that can be seen
pd.options.display.max_colwidth = 120           #just extending the width of columns to be able to click on Url for checking infos
data.shape

### Checking  if there is any duplicates

In [ ]:
data.drop_duplicates("PropertyId",inplace=True)
data.shape



### Seems not let's move on to the next step which  is checking null values

In [ ]:
data.isnull().sum()

### Get rid of null values when possible

In [ ]:
data.update(data[["BathroomCount","Fireplace","Furnished","Garden","GardenArea","SwimmingPool","Terrace","ToiletCount"]].fillna(0))
data.shape

### Get rid of potentials encoding errors

In [ ]:
data.drop(data[data.BathroomCount > data.BedroomCount].index,inplace=True)
data.drop(data[data.ConstructionYear > 2033].index,inplace=True)
data.drop(data[data.GardenArea > data.SurfaceOfPlot].index,inplace=True)
data.drop(data[data.PostalCode < 1000].index,inplace=True)
data.drop(data[data.NumberOfFacades > 4].index,inplace=True)
data.drop(data[data.Price > 15000000].index,inplace=True)
data.drop(data[data.ToiletCount > 58].index,inplace=True)
data.drop(data[data.ShowerCount > 58].index,inplace=True)
data.drop(data[data.Price == data.LivingArea].index,inplace=True)
data.drop(data[data.LivingArea > 8800].index, inplace=True)
data.drop(data[data.TypeOfSale == "annuity_monthly_amount"].index,inplace=True)
data.drop(data[data.TypeOfSale == "annuity_without_lump_sum"].index,inplace=True)
data.drop(data[data.TypeOfSale == "annuity_lump_sum"].index,inplace=True)


data.shape

### Super looks like we deleted more than 4000 errors. Great!

### Separate sale data and rent data to express more things

In [ ]:
sale_data = data[data.TypeOfSale == "residential_sale"]
rent_data = data[data.TypeOfSale == "residential_monthly_rent"]
print(sale_data.shape)
print(rent_data.shape)

## Data Analysis

### See how many rows and columns we have

In [ ]:
print(sale_data.shape)
print(rent_data.shape)

### See correlation between price and place

#### price/region

In [ ]:
price_sale_region = sale_data.groupby("Region", as_index=False)[["Price"]].mean().round()
price_rent_region = rent_data.groupby("Region", as_index=False)[["Price"]].mean().round()

In [ ]:
fig = go.Figure()
colors = px.colors.qualitative.Vivid
fig.add_trace(
    go.Bar(
        x=price_sale_region['Region'],
        y=price_sale_region['Price'],
        name='Sale',
        offsetgroup=1,
        marker=dict(
            color=colors[0]
        )
    )
)
fig.add_trace(
    go.Bar(
        x=price_rent_region['Region'],
        y=price_rent_region['Price'],
        name='Rent',
        yaxis='y2',
        offsetgroup=2,
        marker=dict(
            color=colors[1]
        )
    )
)
fig.update_traces(
    hovertemplate='<b>%{y:,.0f} €</b>'
)
buttons=[
    dict(
        label="Rent",
        method="update",
        args=[{},dict(
            xaxis=dict(
                categoryarray=price_rent_region['Region']
            )   
        )]
    ),
    dict(
        label="Sale",
        method="update",
        args=[{},dict(
            xaxis=dict(
                categoryarray=price_sale_region['Region']
            )   
        )]
    )
]
fig.update_layout(
    title='Average price per region',
    width = 800,
    updatemenus=[dict(
        type='buttons',
        buttons=buttons,
        bgcolor=colors[-1]
    )],
    legend=dict(
        x=0.5,
        y=1.1,
        orientation='h',
        xanchor='center'
    ),
    yaxis=dict(
        gridcolor=colors[0],
        color=colors[0],
        title='Sale price in €'
    ),
    yaxis2=dict(
        overlaying='y',
        side='right',
        gridcolor=colors[1],
        color=colors[1],
        title='Monthly rent in €'
    ),
        xaxis=dict(
        categoryorder='array',
    ),
    bargap=0.1,
    bargroupgap=0.05,
    hovermode='x unified',
)
fig.show()

#### price/province

In [ ]:
price_sale_province = sale_data.groupby("Province", as_index=False)[["Price"]].mean().round().sort_values("Price",ascending=False)
price_rent_province = rent_data.groupby("Province", as_index=False)[["Price"]].mean().round().sort_values("Price",ascending=False)

In [ ]:
fig = go.Figure()
colors = px.colors.qualitative.Vivid
fig.add_trace(
    go.Bar(
        x=price_sale_province['Province'],
        y=price_sale_province['Price'],
        name='Sale',
        offsetgroup=1,
        marker=dict(
            color=colors[0]
        )
    )
)
fig.add_trace(
    go.Bar(
        x=price_rent_province['Province'],
        y=price_rent_province['Price'],
        name='Rent',
        yaxis='y2',
        offsetgroup=2,
        marker=dict(
            color=colors[1]
        )
    )
)
fig.update_traces(
    hovertemplate='<b>%{y:,.0f} €</b>',   
)
buttons=[
    dict(
        label="Sale",
        method="update",
        args=[{},dict(
            xaxis=dict(
                categoryarray=price_sale_province['Province']
            )   
        )]
    ),
    dict(
        label="Rent",
        method="update",
        args=[{},dict(
            xaxis=dict(
                categoryarray=price_rent_province['Province']
            )   
        )]
    )
]
fig.update_layout(
    title='Average price per province',
    width = 800,
    updatemenus=[dict(
        buttons=buttons
    )],
    legend=dict(
        x=0.5,
        y=1.1,
        orientation='h',
        xanchor='center'
    ),
    yaxis=dict(
        gridcolor=colors[0],
        color=colors[0],
        title='Sale price in €'
    ),
    yaxis2=dict(
        overlaying='y',
        side='right',
        gridcolor=colors[1],
        color=colors[1],
        title='Monthly rent in €'
    ),
    xaxis=dict(
        categoryorder='array',
    ),
    bargap=0.1,
    bargroupgap=0.05,
    hovermode='x unified',
)
fig.show()

#### Price/District

In [ ]:
price_sale_district = sale_data.groupby("District", as_index=False)[["Price"]].mean().round().sort_values("Price",ascending=False)
price_rent_district = rent_data.groupby("District", as_index=False)[["Price"]].mean().round().sort_values("Price",ascending=False)

In [ ]:
fig = go.Figure()
colors = px.colors.qualitative.Vivid
fig.add_trace(
    go.Bar(
        x=price_sale_district['District'],
        y=price_sale_district['Price'],
        name='Sale',
        offsetgroup=1,
        marker=dict(
            color=colors[0]
        )
    )
)
fig.add_trace(
    go.Bar(
        x=price_rent_district['District'],
        y=price_rent_district['Price'],
        name='Rent',
        yaxis='y2',
        offsetgroup=2,
        marker=dict(
            color=colors[1]
        )
    )
)
fig.update_traces(
    hovertemplate='<b>%{y:,.0f} €</b>',   
)
buttons=[
    dict(
        label="Sale",
        method="update",
        args=[{},dict(
            xaxis=dict(
                categoryarray=price_sale_district['District']
            )   
        )]
    ),
    dict(
        label="Rent",
        method="update",
        args=[{},dict(
            xaxis=dict(
                categoryarray=price_rent_district['District']
            )   
        )]
    )
]
fig.update_layout(
    title='Average price per district',
    width = 800,
    updatemenus=[dict(
        buttons=buttons
    )],
    legend=dict(
        x=0.5,
        y=1.15,
        orientation='h',
        xanchor='center'
    ),
    yaxis=dict(
        gridcolor=colors[0],
        color=colors[0],
        title='Sale price in €'
    ),
    yaxis2=dict(
        overlaying='y',
        side='right',
        gridcolor=colors[1],
        color=colors[1],
        title='Monthly rent in €'
    ),
    xaxis=dict(
        categoryorder='array',
    ),
    bargap=0.1,
    bargroupgap=0.05,
    hovermode='x unified',
)
fig.show()

### Price per m2

#### Per Region

In [ ]:
region_price = sale_data.groupby("Region")[["Price"]].sum()
region_livingarea = sale_data.groupby("Region")[["LivingArea"]].sum()
region_m2_price = pd.merge(region_price,region_livingarea,on="Region")
region_m2_price["€/m2"] = region_m2_price["Price"] / region_m2_price["LivingArea"]
region_m2_price = region_m2_price.reset_index()
region_m2_price.plot.bar(x="Region",y="€/m2")

#### Per Province

In [ ]:
province_price = sale_data.groupby("Province",as_index=False)[["Price"]].sum()
province_livingarea = sale_data.groupby("Province",as_index=False)[["LivingArea"]].sum()
province_m2_price = pd.merge(province_price,province_livingarea,on="Province")
province_m2_price["€/m2"] = province_m2_price["Price"] / province_m2_price["LivingArea"]
province_m2_price.plot.bar(x="Province",y="€/m2")

#### Per District

In [ ]:
district_price = sale_data.groupby("District",as_index=False)[["Price"]].sum()
district_livingarea = sale_data.groupby("District",as_index=False)[["LivingArea"]].sum()
district_m2_price = pd.merge(district_price,district_livingarea,on="District")
district_m2_price["€/m2"] = district_m2_price["Price"] / district_m2_price["LivingArea"]
district_m2_price.plot.bar(x="District",y="€/m2")

In [ ]:
sale_data.head(1)

In [ ]:
sale_data.StateOfBuilding.value_counts()

In [ ]:
stateofbuilding_to_num = {"StateOfBuilding":{"TO_BE_DONE_UP": 1, "TO_RESTORE" : 2, "TO_RENOVATE" : 3, "GOOD" : 4, "JUST_RENOVATED" : 5, "AS_NEW" : 6}}
sale_data.replace(stateofbuilding_to_num, inplace=True)
peb_sob = sale_data.groupby("PEB", as_index=False)[["StateOfBuilding"]].mean()
peb_sob.dropna(inplace=True)
peb_sob.sort_values("StateOfBuilding",ascending=False)

peb_sob.plot.bar(x="PEB", y="StateOfBuilding")

### €/m2 per municipality

#### adding Nom commune column

In [ ]:
municipality_name = pd.read_excel("Conversion Postal code_Refnis code_va01012019.xlsx")
zip_code = municipality_name[["Postal code", "Nom commune","Refnis code"]]
zip_code.rename(columns={"Postal code": "PostalCode"}, inplace=True)
data_municipality = pd.merge(sale_data,zip_code,on="PostalCode",how='inner')
data_municipality = data_municipality.drop_duplicates("PropertyId")
data_municipality

### More and less expensives municipality in Wallonia

In [ ]:
municipality_price = data_municipality[["Nom commune","Region","Price","LivingArea","Refnis code"]]
wallonia_price = municipality_price[~municipality_price.Region.isin(["Flanders", "Brussels"])]
wallonia_price["€/m2"] = wallonia_price["Price"] / wallonia_price["LivingArea"]
wallonia_price = wallonia_price.groupby("Nom commune",as_index=False)[["€/m2"]].mean()
wallonia_price = wallonia_price.sort_values("€/m2", ascending=False)
print(wallonia_price.head(1))
print(wallonia_price.tail(1))

### More and less expensives municipality in Brussels

In [ ]:
brussels_price = municipality_price[~municipality_price.Region.isin(["Flanders", "Wallonie"])]
brussels_price["€/m2"] = brussels_price["Price"] / brussels_price["LivingArea"]
brussels_price = brussels_price.groupby("Nom commune",as_index=False)[["€/m2"]].mean()
brussels_price = brussels_price.sort_values("€/m2", ascending=False)
print(brussels_price.head(1))
print(brussels_price.tail(1))

### More and less expensives municipality in Flanders

In [ ]:
flanders_price = municipality_price[~municipality_price.Region.isin(["Brussels", "Wallonie"])]
flanders_price["€/m2"] = flanders_price["Price"] / flanders_price["LivingArea"]
flanders_price = flanders_price.groupby("Nom commune",as_index=False)[["€/m2"]].mean()
flanders_price = flanders_price.sort_values("€/m2", ascending=False)
print(flanders_price.head(1))
print(flanders_price.tail(1))

### Global €/m2 per municpality

In [ ]:
municipality_price_m2 = data_municipality[["LivingArea","Price","Nom commune"]]
municipality_price_m2 = data_municipality.groupby("Nom commune",as_index=False)[["LivingArea","Price"]].sum()
municipality_price_m2["Refnis code"] = data_municipality["Refnis code"]
municipality_price_m2["€/m2"] = municipality_price_m2["Price"] / municipality_price_m2["LivingArea"]
municipality_price_m2.sort_values("€/m2",ascending=False)

In [ ]:
municipality_price_m2.plot.scatter(x="Nom commune",y= "€/m2")


### Just for fun the link between url lenght and Price

In [ ]:
sale_data["length_url"] = sale_data["Url"].apply(len)
sale_data = sale_data.groupby("length_url",as_index=False)[["Price"]].mean().sort_values("length_url",ascending=False)
sale_data.plot.bar(x="length_url",y="Price")

